In [1]:
from simpletransformers.classification import *
import sklearn
import pandas as pd
import logging
from util import *
from sklearn.preprocessing import MultiLabelBinarizer
#from transformers imp ort *
import numpy as np
import os
import re
import tensorflow as tf
import torch

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [2]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [3]:
train_df = pd.DataFrame()
train_df["text"] = training_set.abstract
train_df["labels"] = training_set.AnalysisAndModeling.astype(str)

In [4]:
test_df = pd.DataFrame()
test_df["text"] = testing_set.description
test_df['labels'] = testing_set.AnalysisAndModeling.astype(str)

In [5]:
test_df

,text,labels
0,TECHNICAL FIELD \n The present disclosure ...,True
1,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,True
2,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,False
3,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,False
4,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,True
...,...,...
238,CROSS-REFERENCE TO RELATED APPLICATIONS \n T...,False
239,FIELD OF THE INVENTION \n The present inve...,True
240,BACKGROUND OF THE INVENTION \n The present i...,False
241,FIELD \n The present teachings relate to a...,False


In [6]:
model_args = ClassificationArgs()
model_args.fp16 = True
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "f1"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 100
model_args.num_train_epochs = 50
model_args.do_lower_case = False
model_args.dataloader_num_workers=0
model_args.overwrite_output_dir = True
model_args.evaluate_during_training=True
model_args.evaluate_during_training_silent=False
model_args.evaluate_during_training_verbose=True
model_args.use_cached_eval_features=True
model_args.evaluate_each_epoch=False
model_args.evaluate_during_training_steps = 32
model_args.max_seq_length=256
model_args.train_batch_size=32
model_args.save_best_model=True
model_args.save_model_every_epoch=False
model_args.save_eval_checkpoints=False
model_args.labels_list = ["True", "False"]
#model_args.sliding_windows=True

In [8]:
model_args.learning_rate = 4e-04
model_args.warmup_ratio = 0.1

In [9]:
#model = BertForPreTraining.from_pretrained("/var/patentmark/patentBERT")

In [10]:
#model.save_pretrained("/var/patentmark/conv-patentBERT2/")

In [11]:
model = ClassificationModel(
    "albert", "albert-base-v2", args=model_args)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [12]:
model.train_model(train_df, eval_df=test_df, f1=sklearn.metrics.f1_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_albert_256_2_243


/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 84, 'tn': 0, 'fp': 159, 'fn': 0, 'f1': 0.5137614678899083, 'eval_loss': 0.8376387376939097}


/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.classification.classification_model: No improvement in f1
INFO:simpletransformers.classification.classification_model: Current step: 1
INFO:simpletransformers.classification.classification_model: Early stopping patience: 5


INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_albert_256_2_243


...

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_albert_256_2_243


INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 84, 'tn': 0, 'fp': 159, 'fn': 0, 'f1': 0.5137614678899083, 'eval_loss': 0.7288853083887408}
INFO:simpletransformers.classification.classification_model: No improvement in f1
INFO:simpletransformers.classification.classification_model: Current step: 5
INFO:simpletransformers.classification.classification_model: Early stopping patience: 5


INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_albert_256_2_243


INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 84, 'tn': 0, 'fp': 159, 'fn': 0, 'f1': 0.5137614678899083, 'eval_loss': 0.7278599527574354}
INFO:simpletransformers.classification.classification_model: Patience of 5 steps reached
INFO:simpletransformers.classification.classification_model: Training terminated.
INFO:simpletransformers.classification.classification_model: Training of albert model complete. Saved to outputs/.


(192,
 {'global_step': [32, 64, 96, 128, 160, 192],
  'tp': [84, 84, 84, 84, 84, 84],
  'tn': [0, 0, 0, 0, 0, 0],
  'fp': [159, 159, 159, 159, 159, 159],
  'fn': [0, 0, 0, 0, 0, 0],
  'mcc': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'train_loss': [0.6682323813438416,
   0.6601671576499939,
   0.8827059268951416,
   0.6258347034454346,
   0.6931665539741516,
   0.6723867654800415],
  'eval_loss': [0.8376387376939097,
   0.7171270482001766,
   0.8562185985426749,
   0.8974122212779138,
   0.7288853083887408,
   0.7278599527574354],
  'f1': [0.5137614678899083,
   0.5137614678899083,
   0.5137614678899083,
   0.5137614678899083,
   0.5137614678899083,
   0.5137614678899083]})

In [ ]:
from sklearn.metrics import *

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [ ]:
model_outputs

In [ ]:
import numpy as np
np.mean(model_outputs)

In [ ]:
print(classification_report(testing_labels, model_outputs, target_names=mlb.classes_))

In [ ]:
# import numpy as np
# import os
# import re
# import tensorflow as tf
# import torch
# def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file, pytorch_dump_path):
#     config_path = os.path.abspath(bert_config_file)
#     tf_path = os.path.abspath(tf_checkpoint_path)
#     print("Converting TensorFlow checkpoint from {} with config at {}".format(tf_path, config_path))
#     # Load weights from TF model
#     init_vars = tf.train.list_variables(tf_path)
#     excluded = ["BERTAdam", "_power", "global_step", "_CHECKPOINTABLE_OBJECT_GRAPH"]
#     init_vars = list(filter(lambda x: all([True if e not in x[0] else False for e in excluded]), init_vars))
#     names = []
#     arrays = []
#     for name, shape in init_vars:
#         print("Loading TF weight {} with shape {}".format(name, shape))
#         array = tf.train.load_variable(tf_path, name)
#         names.append(name)
#         arrays.append(array)

#     config = BertConfig.from_json_file(bert_config_file)
#     print("Building PyTorch model from configuration: {}".format(str(config)))
#     # Initialise PyTorch model
#     model = BertForSequenceClassification(config)

#     for name, array in zip(names, arrays):
#         name = name.split("/")
#         # adam_v and adam_m are variables used in AdamWeightDecayOptimizer to calculated m and v
#         # which are not required for using pretrained model
#         if any(n in ["adam_v", "adam_m", "global_step", "bad_steps", "global_step", "good_steps", "loss_scale",
#                      "AdamWeightDecayOptimizer", "AdamWeightDecayOptimizer_1", "save_counter", ".OPTIMIZER_SLOT"] for n in name) or \
#                 name[0] == "optimizer":
#             print("Skipping {}".format("/".join(name)))
#             continue
#         if ".OPTIMIZER_SLOT" in name:
#             idx = name.index(".OPTIMIZER_SLOT")
#             name = name[:idx]
#         elif ".ATTRIBUTES" in name:
#             idx = name.index(".ATTRIBUTES")
#             name = name[:idx]
#         print(name)
#         pointer = model
#         for m_name in name:
#             try:
#                 if re.fullmatch(r"[A-Za-z]+_\d+", m_name):
#                     scope_names = re.split(r"_(\d+)", m_name)
#                 else:
#                     scope_names = [m_name]
#                 if scope_names[0] == "kernel" or scope_names[0] == "gamma":
#                     pointer = getattr(pointer, "weight")
#                 elif scope_names[0] == "output_bias" or scope_names[0] == "beta":
#                     pointer = getattr(pointer, "bias")
#                 elif scope_names[0] == "output_weights":
#                     pointer = getattr(pointer, "weight")
#                 elif scope_names[0] == "squad":
#                     pointer = getattr(pointer, "classifier")
#                 elif scope_names[0] == "dense_output" or scope_names[0] == "bert_output":
#                     pointer = getattr(pointer, "output")
#                 elif scope_names[0] == "self_attention":
#                     pointer = getattr(pointer, "self")
#                 else:
#                     pointer = getattr(pointer, scope_names[0])
#             except AttributeError:
#                     logger.info("Skipping {}".format("/".join(name)))
#                     continue
#             if len(scope_names) >= 2:
#                 num = int(scope_names[1])
#                 pointer = pointer[num]
#         if m_name[-11:] == "_embeddings":
#             pointer = getattr(pointer, "weight")
#         elif m_name == "kernel" or m_name == "gamma" or m_name == "output_weights":
#             array = np.transpose(array)
#         # print("Initialize PyTorch weight {}".format(name))
#         pointer.data = torch.from_numpy(array)

#     # Save pytorch-model
#     print("Save PyTorch model to {}".format(pytorch_dump_path))
#     torch.save(model.state_dict(), pytorch_dump_path)


# convert_tf_checkpoint_to_pytorch("/var/patentmark/patentBERT/model.ckpt", "/var/patentmark/patentBERT/config.json", pytorch_dump_path="/var/patentmark/patentBERT/pytorch.bin")